### Importando dataset gerado via webcrawler

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("resultado_pesquisa_2021_07_10_23_49_56.csv", sep=";", encoding = "ISO-8859-1")

In [3]:
dataset.head()

,classe,assunto,magistrado,comarca,foro,vara,data de disponibilizacao,texto
0,Ação Penal de Competência do Júri,Homicídio Simples,Bruno Nascimento Troccoli,Mongaguá,Foro de Mongaguá,2ª Vara,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
1,Ação Penal - Procedimento Sumário,Crimes de Trânsito,FABIO ANTONIO CAMARGO DANTAS,Jales,Foro de Jales,1ª Vara Criminal,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
2,Ação Penal de Competência do Júri,Homicídio Qualificado,EMERSON TADEU PIRES DE CAMARGO,Sorocaba,Foro de Sorocaba,Vara do Júri/Execuções,09/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
3,Ação Penal - Procedimento Ordinário,Furto,EDUARDO DE FRANCA HELENE,São José dos Campos,Foro de São José dos Campos,3ª Vara Criminal,08/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...
4,Ação Penal - Procedimento Ordinário,Crimes de Trânsito,Fabio Pando de Matos,SÃO PAULO,Foro Central Criminal Barra Funda,10ª Vara Criminal,08/07/2021,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO CO...


### Limpando os dados e removendo informações desnecessárias

In [4]:
non_null_ds = dataset.dropna()

cutted_ds = non_null_ds[['assunto','magistrado','texto']].copy()
cutted_ds = cutted_ds[cutted_ds.assunto.str.contains('Homicídio')]
cutted_ds = cutted_ds[cutted_ds.texto.str.contains('Vistos.')]
cutted_ds.texto.head()

0     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
2     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
7     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
9     TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
10    TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO  CO...
Name: texto, dtype: object

In [5]:
cutted_ds['texto'] = cutted_ds['texto'].map(lambda x: str(x).split('Vistos.')[-1].split('Intime-se')[0])